# 네이버 cloud flatform에서 지도 API 제공

In [40]:
import folium
import pandas as pd
import urllib.request
import datetime
import time
import json
from config_NaverAPI_okcy import *    # NAVER Client ID/Secret

In [41]:
import os
import sys
import webbrowser

## 네이버 지도 API-id, secret-key

In [42]:
#[CODE 1]
def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    req.add_header("X-NCP-APIGW-API-KEY", client_secret)
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [55]:
base = "https://openapi.naver.com/v1/search/blog?query="
address = "울산시 남구 대학로 93"
parameters = "?query=%s" % urllib.parse.quote(address)
url = base + parameters
retData = get_request_url(url)

HTTP Error 401: Unauthorized
[2023-03-23 13:51:29.841602] Error for URL : https://openapi.naver.com/v1/search/blog?query=?query=%EC%9A%B8%EC%82%B0%EC%8B%9C%20%EB%82%A8%EA%B5%AC%20%EB%8C%80%ED%95%99%EB%A1%9C%2093


In [53]:
retData

'{"error_code":"052","message":"v2 : Partner does not exists. (등록된 파트너가 없습니다.)"}'

In [54]:
jsonAddress = json.loads(retData)
jsonAddress

{'error_code': '052',
 'message': 'v2 : Partner does not exists. (등록된 파트너가 없습니다.)'}

In [6]:
longitude = jsonAddress['addresses'][0]['x']
latitude = jsonAddress['addresses'][0]['y']
print([latitude, longitude])

NameError: name 'jsonAddress' is not defined

## 검색 주소의 지도 정보를JSON 형식으로 return

In [7]:
#[CODE 2]
def getGeoData(address):
    
    base = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    
    try:
        parameters = "?query=%s" % urllib.parse.quote(address)
    except:
        return None
    
    url = base + parameters
    
    retData = get_request_url(url)
    if retData == None:
        return None

    jsonAddress = json.loads(retData)

    if 'addresses' in jsonAddress.keys():
        if jsonAddress['addresses'] != []:
            latitude = jsonAddress['addresses'][0]['y']
            longitude = jsonAddress['addresses'][0]['x']
        else:
            return None
    else:
        return None
    
    return [latitude, longitude]

## folium map 객체 생성(울산대학교 )

In [8]:
map_uou = folium.Map(location=[35.5437411,129.2540956], zoom_start=12)
map_uou

In [9]:
folium.Marker([35.5437411,129.2540956], popup='울산대학교').add_to(map_uou)
map_uou

## 가맹점 데이터 파일에서 주소

In [10]:
map_chicken = folium.Map(location=[35.5437411,129.2540956], zoom_start=12)
map_chicken

In [12]:
#[CODE 4]
import pandas as pd
import pdb
filename = 'chicken_data/cheogajip_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []

for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='red')).add_to(map_chicken)

HTTP Error 401: Unauthorized
[2023-03-23 13:09:56.184621] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EB%82%A8%EA%B5%AC%20%EB%8C%80%ED%95%99%EB%A1%9C%2094%EB%B2%88%EA%B8%B8%206
HTTP Error 401: Unauthorized
[2023-03-23 13:09:56.287057] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EB%82%A8%EA%B5%AC%20%EC%A4%91%EC%95%99%EB%A1%9C%2038%28%EC%95%BC%EC%9D%8C%EB%8F%99%29
HTTP Error 401: Unauthorized
[2023-03-23 13:09:56.409113] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EB%8F%99%EA%B5%AC%20%EB%8C%80%EC%86%A18%EA%B8%B8%2031%28%ED%99%94%EC%A0%95%EB%8F%99%29
HTTP Error 401: Unauthorized
[2023-03-23 13:09:56.707684] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8

HTTP Error 401: Unauthorized
[2023-03-23 13:10:01.848783] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%20%EB%82%A8%EA%B5%AC%20%EC%95%BC%EC%9D%8C%EB%8F%99%20459-52
HTTP Error 401: Unauthorized
[2023-03-23 13:10:01.919251] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EB%82%A8%EA%B5%AC%20%EC%B2%98%EC%9A%A9%EB%A1%9C%2026%EB%B2%88%EA%B8%B8%2010%28%EC%95%BC%EC%9D%8C%EB%8F%99%29
HTTP Error 401: Unauthorized
[2023-03-23 13:10:01.999486] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EC%9A%B8%EC%A3%BC%EA%B5%B0%20%EC%B2%AD%EB%9F%89%EB%A9%B4%20%EC%82%BC%EC%A0%95%EB%A1%9C%20769
HTTP Error 401: Unauthorized
[2023-03-23 13:10:02.075939] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EC%9A%B8

In [13]:
map_chicken

In [14]:
filename = 'chicken_data/kyochon_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='blue')).add_to(map_chicken)

HTTP Error 401: Unauthorized
[2023-03-23 13:10:09.071999] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%20%EB%82%A8%EA%B5%AC%20%EC%8B%A0%EC%A0%95%EB%8F%99%201218-7
HTTP Error 401: Unauthorized
[2023-03-23 13:10:09.142152] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%20%EB%82%A8%EA%B5%AC%20%EB%8B%AC%EB%8F%99%20135-2
HTTP Error 401: Unauthorized
[2023-03-23 13:10:09.216498] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%20%EB%82%A8%EA%B5%AC%20%EB%AC%B4%EA%B1%B0%EB%8F%99%201484-2
HTTP Error 401: Unauthorized
[2023-03-23 13:10:09.291309] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%20%EB%82%A8%EA%B5%AC%20%EB%8B%AC%EB%8F%99%201331-20
HTTP Error 401: Unauthorized
[2023-03-23 13:10:09.366426] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?que

In [15]:
map_chicken

In [16]:
filename = 'chicken_data/goobne_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='gray')).add_to(map_chicken)

HTTP Error 401: Unauthorized
[2023-03-23 13:10:11.875889] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EB%B6%81%EA%B5%AC%20%20%EC%83%81%EB%B0%A9%EB%A1%9C%20139
HTTP Error 401: Unauthorized
[2023-03-23 13:10:11.951309] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EB%8F%99%EA%B5%AC%20%EC%9D%BC%EC%82%B0%EB%8F%99%20535-4%EB%B2%88%EC%A7%80
HTTP Error 401: Unauthorized
[2023-03-23 13:10:12.036437] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EC%9A%B8%EC%A3%BC%EA%B5%B0%20%EC%98%A8%EC%96%91%EC%9D%8D%20%EB%8C%80%EC%95%88%EB%A6%AC415-19%EB%B2%88%EC%A7%80
HTTP Error 401: Unauthorized
[2023-03-23 13:10:12.113205] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%2

In [17]:
map_chicken

In [18]:
filename = 'chicken_data/nene_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='blacks')).add_to(map_chicken)

HTTP Error 401: Unauthorized
[2023-03-23 13:10:14.006193] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%20%EC%9A%B8%EC%A3%BC%EA%B5%B0%20%EC%98%A8%EC%96%91%EC%9D%8D%20%EC%97%B0%EC%95%884%EA%B8%B818
HTTP Error 401: Unauthorized
[2023-03-23 13:10:14.090720] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%20%EC%9A%B8%EC%A3%BC%EA%B5%B0%20%EB%B2%94%EC%84%9C%EC%9D%8D%20%EB%8C%80%EB%A6%AC1%EA%B8%B823-6
HTTP Error 401: Unauthorized
[2023-03-23 13:10:14.169684] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%20%EB%B6%81%EA%B5%AC%20%EB%8F%99%EB%8C%808%EA%B8%B812
HTTP Error 401: Unauthorized
[2023-03-23 13:10:14.243113] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%20%EC%A4%91%EA%B5%AC%20%EC%9C%A0%EA%B3%A1%EB%A1%9C%2070
HTTP Error 401: Unauthorized
[2023-03-23 13:10:14.322456] Erro

In [19]:
map_chicken

In [20]:
filename = 'chicken_data/pericana_table.csv'
df = pd.read_csv(filename, encoding='CP949', index_col=0, header=0)
geoData = []
for index, row in df.iterrows():
    if row['sido'] == '울산광역시':
        geoData = getGeoData(row['store_address'])
        if geoData != None:
            folium.Marker(geoData, popup=row['store'], icon=folium.Icon(color='Greens')).add_to(map_chicken)

HTTP Error 401: Unauthorized
[2023-03-23 13:10:16.061480] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EB%8F%99%EA%B5%AC%20%EB%8F%99%ED%95%B4%EC%95%88%EB%A1%9C%2040-2%28%EB%8F%99%EB%B6%80%EB%8F%99%29
HTTP Error 401: Unauthorized
[2023-03-23 13:10:16.134470] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EC%9A%B8%EC%A3%BC%EA%B5%B0%20%EC%98%A8%EC%96%91%EC%9D%8D%20%EB%8C%80%EC%95%88%EB%A6%AC%20411-4
HTTP Error 401: Unauthorized
[2023-03-23 13:10:16.316745] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%EB%82%A8%EA%B5%AC%20%EB%8B%AC%EB%8F%99%201329-9
HTTP Error 401: Unauthorized
[2023-03-23 13:10:16.394409] Error for URL : https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C%20%

In [21]:
map_chicken

In [22]:
#[CODE 5]
svFilename = 'chicken_가맹점_울산광역시.html'
map.save(svFilename)
webbrowser.open(svFilename)

AttributeError: type object 'map' has no attribute 'save'